In [1]:
import numpy as np
import cv2

# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')


cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        blurred_image = cv2.GaussianBlur(roi_gray,(5,5),0)
        eyes = eye_cascade.detectMultiScale(blurred_image)
        
    for (ex,ey,ew,eh) in eyes:
         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

ImportError: No module named cv2

In [ ]:
#Identify pupils. Based on beta 1

import numpy as np
import cv2
import time

cap = cv2.VideoCapture(0) #640,480
w = 640
h = 480

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        
        #downsample
        #frameD = cv2.pyrDown(cv2.pyrDown(frame))
        #frameDBW = cv2.cvtColor(frameD,cv2.COLOR_RGB2GRAY)

        #detect face
        frame = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        faces = cv2.CascadeClassifier('haarcascade_eye.xml')
        detected = faces.detectMultiScale(frame, 1.3, 5)

        #faces = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        #detected2 = faces.detectMultiScale(frameDBW, 1.3, 5)

        pupilFrame = frame
        pupilO = frame
        windowClose = np.ones((5,5),np.uint8)
        windowOpen = np.ones((2,2),np.uint8)
        windowErode = np.ones((2,2),np.uint8)

        #draw square
        for (x,y,w,h) in detected:
            cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (0,0,255),1)	
            cv2.line(frame, (x,y), ((x+w,y+h)), (0,0,255),1)
            cv2.line(frame, (x+w,y), ((x,y+h)), (0,0,255),1)
            pupilFrame = cv2.equalizeHist(frame[int(y+(h*.25)):(y+h), x:(x+w)])
            pupilO = pupilFrame
            ret, pupilFrame = cv2.threshold(pupilFrame,55,255,cv2.THRESH_BINARY)   #50 ..nothin 70 is better
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_CLOSE, windowClose)
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_ERODE, windowErode)
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_OPEN, windowOpen)

        #so above we do image processing to get the pupil..
        #now we find the biggest blob and get the centriod

            threshold = cv2.inRange(pupilFrame,250,255)   #get the blobs
            _, contours, hierarchy = cv2.findContours(threshold,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

        #if there are 3 or more blobs, delete the biggest and delete the left most for the right eye
        #if there are 2 blob, take the second largest
        #if there are 1 or less blobs, do nothing
        
            if len(contours) >= 2:
            #find biggest blob
                maxArea = 0
                MAindex = 0   #to get the unwanted frame 
                distanceX = []   #delete the left most (for right eye)
                currentIndex = 0 
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    center = cv2.moments(cnt)
                    #cx,cy = int(center['m10']/center['m00']), int(center['m01']/center['m00'])
                    if center['m00'] != 0:
                        cx = int(center["m10"] / center["m00"])
                        cy = int(center["m01"] / center["m00"])
                    else:
                        cx,cy = 0, 0
                    distanceX.append(cx)
                    if area > maxArea:
                        maxArea = area
                        MAindex = currentIndex
                    currentIndex = currentIndex + 1

                del contours[MAindex]   #remove the picture frame contour
                del distanceX[MAindex]

            eye = 'right'

            if len(contours) >= 2:    #delete the left most blob for right eye
                if eye == 'right':
                    edgeOfEye = distanceX.index(min(distanceX))
                else:
                    edgeOfEye = distanceX.index(max(distanceX))	
                del contours[edgeOfEye]
                del distanceX[edgeOfEye]

            if len(contours) >= 1:   #get largest blob
                maxArea = 0
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    if area > maxArea:
                        maxArea = area
                        largeBlob = cnt

            if len(largeBlob) > 0:
                center = cv2.moments(largeBlob)
                #cx,cy = int(center['m10']/center['m00']), int(center['m01']/center['m00'])
                if center['m00'] != 0:
                    cx = int(center["m10"] / center["m00"])
                    cy = int(center["m01"] / center["m00"])
                else:
                    cx,cy = 0, 0
                cv2.circle(pupilO,(cx,cy),5,255,-1)


        #show picture
        cv2.imshow('frame',pupilO)
        cv2.imshow('frame2',pupilFrame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    #else:
        #break


# Release everything if job is finished
cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml') 
#olho_direito = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')
#olho_esquerdo =  cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray,(5,5),0)
    faces = face_cascade.detectMultiScale(blurred_image, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = blurred_image[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        #blurred_image = cv2.GaussianBlur(roi_gray,(5,5),0)
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
    for (ex,ey,ew,eh) in eyes:
         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
olho_direito = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')
olho_esquerdo =  cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        #blurred_image = cv2.GaussianBlur(roi_gray,(5,5),0)
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
    for (ex,ey,ew,eh) in eyes:
         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import cv2

# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml') 
#olho_direito = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')
#olho_esquerdo =  cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #blurred_image = cv2.GaussianBlur(gray,(5,5),0)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
    for (ex,ey,ew,eh) in eyes:
         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img',img
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
#import OpenCV library
import cv2
#import matplotlib library
#import matplotlib.pyplot as plt
#importing time library for speed comparisons of both classifiers
import time 
#%matplotlib inline

def convertToRGB(img): 
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#load test iamge
test1 = cv2.imread('data/test1.jpg')
#convert the test image to gray image as opencv face detector expects gray images 
gray_img = cv2.cvtColor(test1, cv2.COLOR_BGR2GRAY)

#if you have matplotlib installed then  
#plt.imshow(gray_img, cmap='gray')  


#load cascade classifier training file for haarcascade 
haar_face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#let's detect multiscale (some images may be closer to camera than others) images 
faces = haar_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5);  

#print the number of faces found 
print('Faces found: ', len(faces))

#go over list of faces and draw them as rectangles on original colored img 
for (x, y, w, h) in faces:     
    cv2.rectangle(test1, (x, y), (x+w, y+h), (0, 255, 0), 2)

# or display the gray image using OpenCV 
cv2.imshow('Test Imag', convertToRGB(test1)) 
cv2.waitKey(0) 
cv2.destroyAllWindows()

def detect_faces(f_cascade, colored_img, scaleFactor = 1.1):
 #just making a copy of image passed, so that passed image is not changed 
    img_copy = colored_img.copy()          

 #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)          

 #let's detect multiscale (some images may be closer to camera than others) images
    faces = f_cascade.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=5);          

 #go over list of faces and draw them as rectangles on original colored img
    for (x, y, w, h) in faces:
        cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)              
    return img_copy

#load another image 
test2 = cv2.imread('data/test3.jpg')  

#call our function to detect faces 
faces_detected_img = detect_faces(haar_face_cascade, test2)  

# or display the gray image using OpenCV 
cv2.imshow('Test Imag', convertToRGB(faces_detected_img)) 
cv2.waitKey(0) 
cv2.destroyAllWindows()

ImportError: No module named cv2

In [ ]:
#import OpenCV library
import cv2
#import matplotlib library
#import matplotlib.pyplot as plt
#importing time library for speed comparisons of both classifiers
import time 
#%matplotlib inline

def convertToRGB(img): 
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def detect_faces(f_cascade, colored_img, scaleFactor = 1.4):
 #just making a copy of image passed, so that passed image is not changed 
    img_copy = colored_img.copy()          

 #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)          

 #let's detect multiscale (some images may be closer to camera than others) images
    faces = f_cascade.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=5);          

 #go over list of faces and draw them as rectangles on original colored img
    for (x, y, w, h) in faces:
        cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)
        eyes_detected_img = detect_eyes(haar_eye_cascade, img_copy) 
    return eyes_detected_img

def detect_eyes(e_cascade, colored_img, scaleFactor = 1.4):
 #just making a copy of image passed, so that passed image is not changed 
    img_copy = colored_img.copy()          

 #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)          

 #let's detect multiscale (some images may be closer to camera than others) images
    eyes = e_cascade.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=5);          

 #go over list of faces and draw them as rectangles on original colored img
    for (x, y, w, h) in eyes:
        cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)              
    return img_copy


#load another image 
test2 = cv2.imread('data/test1.jpg')
 

#load cascade classifier training file for haarcascade 
haar_face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
haar_eye_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')

#call our function to detect faces 
faces_detected_img = detect_faces(haar_face_cascade, test2)    


# or display the gray image using OpenCV 
cv2.imshow('Test Imag', convertToRGB(faces_detected_img)) 
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml') 
#olho_direito = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')
#olho_esquerdo =  cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #blurred_image = cv2.GaussianBlur(gray,(5,5),0)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
       
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.35, 5)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            roi_gray2 = gray[ey:ey+eh, ex:ex+ew]
            roi_color2 = img[ey:ey+eh, ex:ex+ew]
            retval, thresholded = cv2.threshold(roi_gray2, 30, 255, cv2.THRESH_BINARY)
            cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
            circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                            param1=55,param2=50,minRadius=0,maxRadius=0)
            circles = np.uint16(np.around(circles))
            for i in circles[0,:]:
                cv2.circle(cimg,(i[0],i[1]),i[2],(255,0,0),2)
                cv2.circle(cimg,(i[0],i[1]),2,(255,0,0),3)



    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()